# Install packages using pip

In [ ]:
# Install a pip package in the current Jupyter kernel
import sys
# !{sys.executable} -m pip install hszinc
# !{sys.executable} -m pip install pyhaystack
# !{sys.executable} -m pip install pandas
# !{sys.executable} -m pip install matplotlib
# !{sys.executable} -m pip install requests

# Import modules and start session

In [1]:
import hszinc
import pyhaystack
from pyhaystack.client.niagara import Niagara4HaystackSession
from pyhaystack.util import filterbuilder as fb
import pandas as pd
import matplotlib.pyplot as plt
import math
from datetime import date, datetime, timedelta
import requests
import json

# Configure pandas options
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

uri = 'http://175.45.115.115'
username = 'admin1'
password = 'Enviroman1'

# uri = 'http://120.151.12.65'
# username = 'dhara'
# password = 'Kakkad123'

session = Niagara4HaystackSession(uri=uri, username=username, password=password, pint=True)

# Examples

## About
session.about() queries basic information about the server.

In [2]:
op = session.about()
op.wait()
about = op.result

print(about)

<Grid>
	Version: 2.0
	Columns:
		productUri
		tz
		moduleName
		serverName
		productName
		haystackVersion
		productVersion
		moduleVersion
		serverTime
		moduleUri
		serverBootTime
	Row    0:
	productUri=Uri('http://www.tridium.com/')
	tz='Sydney'
	moduleName='nhaystack'
	serverName='SCP'
	productName='Niagara AX'
	haystackVersion='2.0'
	productVersion='4.3.58.18.1'
	moduleVersion='2.0.1'
	serverTime=datetime.datetime(2018, 4, 29, 9, 53, 20, 686000, tzinfo=<DstTzInfo 'Australia/Sydney' AEST+10:00:00 STD>)
	moduleUri=Uri('https://bitbucket.org/jasondbriggs/nhaystack')
	serverBootTime=datetime.datetime(2018, 4, 9, 21, 10, 38, 629000, tzinfo=<DstTzInfo 'Australia/Sydney' AEST+10:00:00 STD>)
</Grid>


## Navigation
session.nav() lets you navigate the structure of the Project Haystack server.

In [ ]:
nav_id = ''
# nav_id = 'his:/'
# nav_id = 'his:/Bourke_Rd_170'

if nav_id:
    op = session.nav(nav_id=nav_id)
    op.wait()
    nav = op.result
else:
    op = session.nav()
    op.wait()
    nav = op.result

print(nav)

## Find Entity
find_entity expects a filter expression, and performs a read specifying the given string as the filter argument

Parameters: <br/>
`filter_expr` – The filter expression to search for. <br/>
`limit` – Optional limit to number of entities retrieved. <br/>
`single` – Are we expecting a single entity? Defaults to True if ids is not a list. <br/>
`callback` – Asynchronous result callback.

In [ ]:
filter_expr = 'site'
limit = 5
    
op = session.find_entity(filter_expr=filter_expr, limit=limit)
op.wait()
result = op.result

print(result)

## Prints the site name and the tags & values for that site

In [ ]:
filter_expr = 'site'
limit = 5

op = session.find_entity(filter_expr=filter_expr, limit=limit)
op.wait()
result = op.result

for site in result.keys():
    print(site + ":")
    el = result[site]
    for tag in el.tags:
        print("\t" + tag + ": " + str(el.tags[tag]))
    print("\n")

## Prints the point name and the tags & values within that point

In [ ]:
filter_expr = 'point'
limit = 100

op = session.find_entity(filter_expr=filter_expr, limit=limit)
op.wait()
result = op.result

for point in result.keys():
    print(point + ":")
    el = result[point]
    for tag in el.tags:
        print("\t" + tag + ": " + str(el.tags[tag]))
    print("\n")

## Filter for 'his' and 'point'
Uses a filter expression to read all the points that have a 'his' tag. <br/>

In [ ]:
filter_expr = 'his and point'
limit = 1000

op = session.find_entity(filter_expr=filter_expr, limit=limit)
op.wait()
result = op.result

for key in result.keys():
    print(key)

## Reads Historical Data into a Series
### Reads the historical data of the given point and returns it as a series.

Parameters: <br/>
`point` – Haystack ‘point’ entity to read the data from <br/>
`rng` – Historical read range for the ‘point’ <br/>
`tz` – Optional timezone to translate timestamps to <br/>
`series_format` – Optional desired format for the series <br/>

In [ ]:
point = 'S.The_Bond_30.G_WCPAC_3.G_WCPAC_3_SA_Temp'
rng = 'today'
tz = None
series_format = None

# rng = 'yesterday'
# rng = '2018-01-01'
# rng = '2018-01-01, 2018-01-07'

op = session.his_read_series(point, rng, tz=tz, series_format=series_format)
op.wait()
result = op.result

print(result.head())
print(result.tail())

## Reads Historical Data into a Grid
### Reads the historical data of the given point and returns it as a hszinc.grid.Grid.

Parameters: <br />
`point` – Haystack ‘point’ entity or ID to read the data from <br />
`rng` – Historical read range for the ‘point’

In [ ]:
point = 'S.The_Bond_30.G_WCPAC_3.G_WCPAC_3_SA_Temp'
rng = 'today'

op = session.his_read(point, rng)
op.wait()
result = op.result

print(result)

## Reads Historical Data into a Dataframe
### Reads the historical data of multiple given points and returns them as a dataframe.

Parameters: <br/>
`columns` – A list of Haystack ‘point’ instances or a dict mapping the column label to the Haystack ‘point’ instance. <br/>
`rng` – Historical read range for the ‘point’ <br/>
`tz` – Optional timezone to translate timestamps to <br/>
`frame_format` – Optional desired format for the data frame

In [ ]:
filter_expr = 'point and his'
limit = 10

columns = []
rng = 'today'
tz = None
frame_format = None

# Reads 10 points to view the history of
op = session.find_entity(filter_expr=filter_expr, limit=limit)
op.wait()
columns = op.result

### NOTE: Some of the points return the errror 'java.lang.ArrayIndexOutOfBoundsException: 0' causing this
### cell to fail, comment out the code below to test with working points 

columns = [ 'C.Drivers.NiagaraNetwork.The_Bond_30_LB2.points.BacnetNetwork.B2_S1_Spy_1.LB1_AC_S2_Zone_Temp',  
            'C.Drivers.NiagaraNetwork.The_Bond_30_LB2.points.BacnetNetwork.B2_S1_Spy_1.LB2_AC_S2_Zone_Temp',  
            'C.Drivers.NiagaraNetwork.The_Bond_30_LB2.points.BacnetNetwork.B2_S1_Spy_1.LB1_AC_S1_Zone_Temp']
               
# Reads the history of the points in columns into a dataframe
op = session.his_read_frame(columns, rng=rng, tz=tz, frame_format=frame_format)
op.wait()
result = op.result

print(result)

# Testing

In [ ]:
filter_expr = 'equip'
limit = 10

op = session.find_entity(filter_expr)
op.wait()
result = op.result

for key in result.keys():
    print(key)

In [ ]:
filter_expr = 'point and his'
limit = 10000

op = session.find_entity(filter_expr=filter_expr, limit=limit)
op.wait()
result = op.result

for point in result.keys():
    print(point + ":")
    el = result[point]
    for tag in el.tags:
        print("\t" + tag + ": " + str(el.tags[tag]))
    print("\n")

In [5]:
filter_expr = 'point and his'
limit = 10000

op = session.find_entity(filter_expr=filter_expr, limit=limit)
op.wait()
result = op.result

for key in result:
    print(key)
    
# S.The_Bond_30.G_WCPAC_3.G_WCPAC_3_Zone_Temp_SP

S.SCP.Ground_AC_Unit_1.Fan_Enable
S.SCP.Ground_AC_Unit_1.Zone_Temp
S.SCP.Ground_AC_Unit_1.Fan_Status
S.SCP.Ground_AC_Unit_1.Schedule
S.SCP.Ground_AC_Unit_1.Cmp_Enable
S.SCP.Ground_AC_Unit_1.Supply_Air_Temp
C.Drivers.NiagaraNetwork.Bourke_Rd_184.points.Ground_AC_Unit_2.Fan_Enable
C.Drivers.NiagaraNetwork.Bourke_Rd_184.points.Ground_AC_Unit_2.Zone_Temp
C.Drivers.NiagaraNetwork.Bourke_Rd_184.points.Ground_AC_Unit_2.Cmp_Enable
C.Drivers.NiagaraNetwork.Bourke_Rd_184.points.Ground_AC_Unit_2.Supply_Air_Temp
C.Drivers.NiagaraNetwork.Bourke_Rd_184.points.Ground_AC_Unit_3.Fan_Enable
C.Drivers.NiagaraNetwork.Bourke_Rd_184.points.Ground_AC_Unit_3.Zone_Temp
C.Drivers.NiagaraNetwork.Bourke_Rd_184.points.Ground_AC_Unit_3.Rev_Valve
C.Drivers.NiagaraNetwork.Bourke_Rd_184.points.Ground_AC_Unit_3.After_Hours
C.Drivers.NiagaraNetwork.Bourke_Rd_184.points.Ground_AC_Unit_3.Cmp_Enable
C.Drivers.NiagaraNetwork.Bourke_Rd_184.points.Ground_AC_Unit_3.Supply_Air_Temp
C.Drivers.NiagaraNetwork.Bourke_Rd_184.points.

In [ ]:
filter_expr = 'point and his and equipRef'
limit = 100

op = session.find_entity(filter_expr=filter_expr, limit=limit)
op.wait()
result = op.result

for key in result:
    equipRef = result[key].tags['equipRef']
    if(str(equipRef) == '@S.The_Bond_30.G_WCPAC_1'):
        print(key)

### Compare data from S.The_Bond_30.G_WCPAC_1

In [ ]:
# Fetches the zone temp set point, the run status, and the return air temp for the specified date range. 

columns = [ 'S.The_Bond_30.G_WCPAC_1.G_WCPAC_1_Zone_Temp_SP',
            'S.The_Bond_30.G_WCPAC_1.G_WCPAC_1_Run_St',
            'S.The_Bond_30.G_WCPAC_1.G_WCPAC_1_RA_Temp']

rng = '2018-02-01, 2018-02-07'

op = session.his_read_frame(columns, rng)
op.wait()
result = op.result

result.head()

In [ ]:
# The timestamps for each column don't match up so the data is re-sampled at 15 minute intervals.
# the run status is a bool value that is only recorded when changed, therefore we must fill in the correct 
# value for each time stamp using the fillna method.

resampledResult = result.resample('15T').agg({
    'S.The_Bond_30.G_WCPAC_1.G_WCPAC_1_Zone_Temp_SP': 'mean',
    'S.The_Bond_30.G_WCPAC_1.G_WCPAC_1_RA_Temp':'mean',
    'S.The_Bond_30.G_WCPAC_1.G_WCPAC_1_Run_St':'last'
})

resampledResult['S.The_Bond_30.G_WCPAC_1.G_WCPAC_1_Run_St'].fillna(method='ffill', inplace=True)

# Replaces the remaining NaNs with False, in this case this is correct 
resampledResult['S.The_Bond_30.G_WCPAC_1.G_WCPAC_1_Run_St'].fillna(False, inplace=True)

# Alternatively, we could just drop the remaining NaNs
# resampledResult.dropna(inplace=True)

resampledResult.head()

In [ ]:
# The re-sampled data is plotted on a graph.
# The run status is converted to a int and the SP temp is added to it so we can see when it is on and off
# compared to the other temps

plotResults = resampledResult.copy()
plotResults['S.The_Bond_30.G_WCPAC_1.G_WCPAC_1_Run_St'] = plotResults['S.The_Bond_30.G_WCPAC_1.G_WCPAC_1_Run_St'].astype(int) + plotResults['S.The_Bond_30.G_WCPAC_1.G_WCPAC_1_Zone_Temp_SP']
plotResults.plot(figsize=(16, 8))

### Compare data from S.The_Bond_30.G_WCPAC_3 and historical weather data

In [ ]:
# Fetches the zone temp set point, the supply air temp, and the return air temp for the specified date range. 

columns = [ 'S.The_Bond_30.G_WCPAC_3.G_WCPAC_3_Zone_Temp_SP',
            'S.The_Bond_30.G_WCPAC_3.G_WCPAC_3_SA_Temp',
            'S.The_Bond_30.G_WCPAC_3.G_WCPAC_3_RA_Temp']

rng = '2018-01-27, 2018-02-10'

op = session.his_read_frame(columns, rng)
op.wait()
result = op.result

result.head()

In [ ]:
# The timestamps for each column don't match up so the data is re-sampled at 15 minute intervals.

resampledResult = result.resample('15T').mean()
resampledResult.head()

In [ ]:
# Initial plot of the temps for the specified date range

resampledResult.plot(figsize=(16, 8))

In [ ]:
# Using the Dark Sky API we can retrieve historical weather data for the site.
# We construct a url from the latitude and longitude of the site and the specified date ranges.

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)
        
lat = -33.8618
lon = 151.2031
api_key = '801e57289de2da50cf9e496e4c823e6c'

start_date = datetime(2018,1,27,0,0)
end_date = datetime(2018,2,10,0,0)

weatherData = pd.DataFrame()

# For the given dates we loop through each day and make an API request to retrieve the historical weather data
# for that day. Each days data is appended to the weatherData dataframe. 
for single_date in daterange(start_date, end_date):
    time = int(single_date.timestamp())
    url = 'https://api.darksky.net/forecast/' + api_key + '/' + str(lat) + ',' + str(lon) + ',' + str(time) + '?units=si'
    r = requests.get(url)
    json = r.json()
    weatherData = weatherData.append(pd.DataFrame(json['hourly']['data']), ignore_index=True)

weatherData.head()

In [ ]:
# The time column is converted from unix time to a datatime object.

weatherData['time'] = pd.to_datetime(weatherData['time'], unit='s', utc=True)

In [ ]:
# Discard other columns leaving the time and temperature columns and set the index as the timestamp.
# Upsample the weather data from every hour to every 15 minutes and interpolate the missing values.

weatherData = weatherData[['time', 'temperature']]
weatherData = weatherData.set_index('time')
weatherData = weatherData.resample('15T').mean()
weatherData = weatherData.interpolate(method='linear')
weatherData.head()

In [ ]:
# Merge the data pulled from Niagara and the historical weather on the indexes (timestamps), convert
# the timezone to Sydney, and rename the columns.

final = resampledResult.merge(weatherData, left_index=True, right_index=True)
final = final.tz_convert('Australia/Sydney')
final.columns = ['SA_Temp', 'Zone_Temp_SP', 'RA_Temp', 'Hist_Weather']
final.head()

In [ ]:
# Plot the Niagara data against the historical weather data.

final.plot(figsize=(16, 8))

In [8]:
# 'C.Drivers.NiagaraNetwork.Oriordan_St_93_East.points.Level_2.FCU_L2_WS.Zone_Temp'
# 'C.Drivers.NiagaraNetwork.Oriordan_St_93_East.points.Level_2.FCU_L2_WS.SAT'
# 'C.Drivers.NiagaraNetwork.Oriordan_St_93_East.points.Level_2.FCU_L2_WS.Fan_Status'
# 'C.Drivers.NiagaraNetwork.Oriordan_St_93_East.points.Level_2.FCU_L2_WS.CHWV'
# 'C.Drivers.NiagaraNetwork.Oriordan_St_93_East.points.Level_2.FCU_L2_WS.HWV'


point = 'C.Drivers.NiagaraNetwork.Oriordan_St_93_East.points.Level_2.FCU_L2_WS.Fan_Status'
# rng = 'today'
tz = None
series_format = None

# rng = 'yesterday'
# rng = '2018-01-01'
# rng = '2018-01-01, 2018-01-07'
rng = '2018-03-01, 2018-03-08'

op = session.his_read_series(point, rng, tz=tz, series_format=series_format)
op.wait()
result = op.result

print(result.to_json())

{"1519844056336":true,"1519887266730":false,"1519930458858":true,"1519973654511":false,"1520189651990":true,"1520232850803":false,"1520276044759":true,"1520319609869":false,"1520362814055":true,"1520406011280":false,"1520449210593":true,"1520492900895":false}
